<a href="https://colab.research.google.com/github/lingo-mit/6864-hw1/blob/master/6864_hw1b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit 
rm -rf 6864-hw1
git clone https://github.com/lingo-mit/6864-hw1.git

In [0]:
import sys
sys.path.append("/content/6864-hw1")

import csv
import itertools as it
import numpy as np
np.random.seed(0)

import lab_util

## Hidden Markov Models

In the remaining part of the lab (containing part 3) you'll use the Baum--Welch algorithm to learn _categorical_ representations of words in your vocabulary. Answers to questions in this lab should go in the same report as the initial release.

As before, we'll start by loading up a dataset:

In [0]:
data = []
n_positive = 0
n_disp = 0
with open("/content/6864-hw1/reviews.csv") as reader:
  csvreader = csv.reader(reader)
  next(csvreader)
  for id, review, label in csvreader:
    label = int(label)

    # hacky class balancing
    if label == 1:
      if n_positive == 2000:
        continue
      n_positive += 1
    if len(data) == 4000:
      break

    data.append((review, label))
    
    if n_disp > 5:
      continue
    n_disp += 1
    print("review:", review)
    print("rating:", label, "(good)" if label == 1 else "(bad)")
    print()

print(f"Read {len(data)} total reviews.")
np.random.shuffle(data)
reviews, labels = zip(*data)
train_reviews = reviews[:3000]
train_labels = labels[:3000]
val_reviews = reviews[3000:3500]
val_labels = labels[3000:3500]
test_reviews = reviews[3500:]
test_labels = labels[3500:]

Next, implement the forward--backward algorithm for HMMs like we saw in class.

**IMPORTANT NOTE**: if you directly multiply probabilities as shown on the class slides, you'll get underflow errors. You'll probably want to work in the log domain (remember that `log(ab) = log(a) + log(b)`, `log(a+b) = logaddexp(a, b)`).

In [0]:
# hmm model
class HMM(object):
    def __init__(self, num_states, num_words):
        self.num_states = num_states
        self.num_words = num_words

        self.states = range(num_states)
        self.symbols = range(num_words)

        # initialize the matrix A with random transition probabilities p(j|i)
        # A should be a matrix of size `num_states x num_states`
        # with rows that sum to 1
        self.A = None # your code here

        # initialize the matrix B with random emission probabilities p(o|i)
        # B should be a matrix of size `num_states x num_words`
        # with rows that sum to 1
        self.B = None # your code here

        # initialize the vector pi with a random starting distribution
        # pi should be a vector of size `num_states`
        self.pi = None # your code here

    def generate(self, n):
        """randomly sample the HMM to generate a sequence.
        """
        # we'll give you this one

        sequence = []
        # initialize the first state
        state = np.random.choice(self.states, p=self.pi)
        for i in range(n):
            # get the emission probs for this state
            b = self.B[state, :]
            # emit a word
            word = np.random.choice(self.symbols, p=b)
            sequence.append(word)
            # get the transition probs for this state
            a = self.A[state, :]
            # update the state
            state = np.random.choice(self.states, p=a)
        return sequence

    def forward(self, obs):
        # run the forward algorithm
        # this function should return a `len(obs) x num_states` matrix
        # where the (i, j)th entry contains p(obs[:t], hidden_state_t = i)

        alpha = np.zeros(len(obs), self.num_states)

        # your code here!

        return alpha

    def backward(self, obs):
        # run the backward algorithm
        # this function should return a `len(obs) x num_states` matrix
        # where the (i, j)th entry contains p(obs[t+1:] | hidden_state_t = i)

        beta = np.zeros(len(obs), self.num_states)

        # your code here!

        return beta
        
    def forward_backward(self, obs):
        # compute forward--backward scores

        # logprob is the total log-probability of the sequence obs 
        # (marginalizing over hidden states)

        # gamma is a matrix of size `len(obs) x num_states`
        # it contains the marginal probability of being in state i at time t

        # xi is a tensor of size `len(obs) x num_states x num_states`
        # it conains the marginal probability of transitioning from i to j at t

        logprob = None
        xi = None
        gamma = None
        # your code here!

        return logprob, xi, gamma

    def learn_unsupervised(self, corpus, num_iters):
        """Run the Baum Welch EM algorithm
        """

        for i_iter in range(num_iters):
            expected_si = None # your code here
            expected_sij = None # your code here
            total_logprob = 0
            for review in corpus:
                logprob, xi, gamma = self.forward_backward(review)
                # your code here 
            print("log-likelihood", total_logprob)
            A_new = None # your code here
            B_new = None # your code here

            self.A = A_new
            self.B = B_new

Train a model:

In [0]:
tokenizer = lab_util.Tokenizer()
tokenizer.fit(train_reviews)
train_reviews_tk = tokenizer.tokenize(train_reviews)
print(tokenizer.vocab_size)

hmm = HMM(num_states=10, num_words=tokenizer.vocab_size)
hmm.learn_unsupervised(train_reviews_tk, 10)

Let's look at some of the words associated with each hidden state:

In [0]:
for i in range(hmm.num_states):
    most_probable = np.argsort(hmm.B[i, :])[:10]
    print(f"state {i}")
    for o in most_probable:
        print(tokenizer.token_to_word[o], hmm.B[i, o])
    print()

We can also look at some samples from the model!

In [0]:
for i in range(10):
    print(tokenizer.de_tokenize([hmm.generate(10)]))

Finally, let's repeat the classification experiment from Parts 1 and 2, using the _vector of expected hidden state counts_ as a sentence representation.

(Warning! results may not be the same as in earlier versions of this experiment.)

In [0]:
def train_model(xs_featurized, ys):
  import sklearn.linear_model
  model = sklearn.linear_model.LogisticRegression()
  model.fit(xs_featurized, ys)
  return model

def eval_model(model, xs_featurized, ys):
  pred_ys = model.predict(xs_featurized)
  print("test accuracy", np.mean(pred_ys == ys))

def training_experiment(name, featurizer, n_train):
    print(f"{name} features, {n_train} examples")
    train_xs = np.array([
        hmm_featurizer(tokenizer.tokenize(review)) 
        for review in train_reviews[:n_train]
    ])
    train_ys = train_labels[:n_train]
    test_xs = np.array([
        hmm_featurizer(tokenizer.tokenize(review))
        for review in test_reviews
    ])
    test_ys = test_labels
    model = train_model(train_xs, train_ys)
    eval_model(model, test_xs, test_ys)
    print()

def hmm_featurizer(review):
    _, _, gamma = hmm.forward_backward(review)
    return gamma.sum(axis=0)

training_experiment("hmm", hmm_featurizer, n_train=100)

**Part 3: Lab writeup**

1. What do the learned hidden states seem to encode when you run unsupervised 
   HMM training with only 2 states? What about 10? What about 100?

2. As before, what's the relationship between # of labeled examples and    
   usefulness of HMM-based sentence representations? Are these results generally
   better or worse than in Parts 1 and 2 of the homework? Why or why not might
   HMM state distributions be sensible sentence representations?